In [1]:
import gdown

url = 'https://drive.google.com/uc?id=1moaGNpmZjjN0XH0dcTOybEz8NVS00x-6'
output = 'train_dataset.txt'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=19ajg9r-QMvDBuVDxBne4Y-vIzSqauAHS'
output = 'test_dataset.txt'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1moaGNpmZjjN0XH0dcTOybEz8NVS00x-6
To: /content/train_dataset.txt
100%|██████████| 17.0M/17.0M [00:00<00:00, 135MB/s] 
Downloading...
From: https://drive.google.com/uc?id=19ajg9r-QMvDBuVDxBne4Y-vIzSqauAHS
To: /content/test_dataset.txt
100%|██████████| 4.25M/4.25M [00:00<00:00, 185MB/s]


'test_dataset.txt'

In [2]:
import csv
from scipy.sparse import csr_matrix

num_records = 0
record = []
with open('train_dataset.txt', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_ALL, quotechar='"')
    next(reader) # Skip the first line
    for row in reader:
        if len(row) == 4:

            record.append(row)

In [3]:
print(len(record))

23038


In [4]:
# Get unique users and items
unique_users = list(set([row[0] for row in record]))
unique_items = list(set([row[1] for row in record]))

# Create mappings from user_id and item_id to row and column indices
user_mapping = {user: index for index, user in enumerate(unique_users)}
item_mapping = {item: index for index, item in enumerate(unique_items)}

# Map user_id and item_id to row and column indices
record = [[user_mapping[row[0]], item_mapping[row[1]], row[2], row[3]] for row in record]

# Separate the ratings and review texts
ratings = [float(row[2]) for row in record]

# Create a sparse matrix for the ratings
r = csr_matrix((ratings, ([record[i][0] for i in range(len(record))], [record[i][1] for i in range(len(record))])))
R_matrix = r.toarray()
print(f'Number of records: {num_records}')
print(f'R matrix: \n{R_matrix}')

Number of records: 0
R matrix: 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
print(r)
print(R_matrix.size)

  (0, 131)	1.0
  (0, 167)	1.0
  (0, 176)	1.0
  (0, 181)	1.0
  (0, 187)	1.0
  (0, 188)	1.0
  (0, 218)	1.0
  (0, 234)	1.0
  (0, 293)	1.0
  (0, 308)	1.0
  (0, 312)	1.0
  (0, 359)	1.0
  (0, 364)	1.0
  (0, 416)	1.0
  (0, 423)	1.0
  (0, 434)	1.0
  (0, 452)	1.0
  (0, 488)	1.0
  (0, 498)	1.0
  (0, 511)	1.0
  (0, 541)	1.0
  (0, 548)	1.0
  (0, 573)	1.0
  (0, 581)	1.0
  (0, 585)	1.0
  :	:
  (1338, 666)	1.0
  (1338, 724)	1.0
  (1339, 4)	1.0
  (1339, 10)	1.0
  (1339, 64)	1.0
  (1339, 119)	1.0
  (1339, 124)	1.0
  (1339, 141)	1.0
  (1339, 150)	1.0
  (1339, 154)	1.0
  (1339, 196)	1.0
  (1339, 201)	1.0
  (1339, 216)	1.0
  (1339, 253)	1.0
  (1339, 300)	1.0
  (1339, 391)	1.0
  (1339, 440)	1.0
  (1339, 441)	1.0
  (1339, 450)	1.0
  (1339, 508)	1.0
  (1339, 509)	1.0
  (1339, 531)	1.0
  (1339, 572)	1.0
  (1339, 610)	1.0
  (1339, 689)	1.0
982220


In [6]:
print(len(unique_users))
print(len(unique_items))

1340
733


In [7]:
print(len(record))

23038


In [8]:
display(R_matrix)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

**Load  BERT model**

In [9]:
from transformers import BertModel, BertTokenizer
import torch


model_name = 'bert-base-uncased'

# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

**:**


In [10]:
#Extract the review texts from the record

reviews = [row[3] for row in record]


# **Generate embeddings**

In [11]:
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

item_ids = [row[1] for row in record]
item_embeddings = {}
for item_id in tqdm(set(item_ids)):
    # Get the indices of the reviews that belong to the current item
    review_indices = [i for i, id in enumerate(item_ids) if id == item_id]
    # Get the review texts that belong to the current item
    item_reviews = [reviews[i] for i in review_indices]
    # Tokenize the review texts and convert them to tensors
    inputs = tokenizer(item_reviews, return_tensors='pt', padding=True, truncation=True).to(device)

    with torch.no_grad():
        outputs = model(**inputs)


    # embedding for each review
    review_embeddings  = outputs.last_hidden_state.mean(dim=1)

    # average of the embeddings for the current item
    item_embeddings[item_id] = review_embeddings.mean(dim=0)
    del inputs, outputs
    torch.cuda.empty_cache()


100%|██████████| 733/733 [12:58<00:00,  1.06s/it]


In [13]:
# Save the item embeddings for next parts

torch.save(item_embeddings, 'item_embeddings.pt')